In [10]:
# Get rid of for blog
# import sys
# sys.path.remove('/Library/Python/2.7/site-packages')

In [11]:
# Do this at the beginning of all your notebooks and you'll never be annoyed with matplotlib formatting again!
import matplotlib as mpl
mpl_update = {'font.size':16,'xtick.labelsize':14,'ytick.labelsize':14,'figure.figsize':[12.0,8.0],
              'axes.color_cycle':['#0055A7', '#2C3E4F', '#26C5ED', '#00cc66', '#D34100', '#FF9700','#091D32'], 
              'axes.labelsize':20,'axes.labelcolor':'#677385','axes.titlesize':20,'lines.color':'#0055A7',
              'lines.linewidth':3,'text.color':'#677385'}
mpl.rcParams.update(mpl_update)

# Trainspotting: real-time detection of a train’s passing from video

Here at Silicon Valley Data Science, we have a slight obsession with the Caltrain. At our last office, we were in an old building directly next door to the Sunnyvale Caltrain station. Everytime a Caltrain passed, the horns blew and we would have to pause whatever we were doing. Now we are just as close to the Mountain View station but are continually thankful for soundproof windows and walls. A quarter of the company relies on the Caltrain to get to work each day and the game of "when will I get to work today?" has become an office favorite; we even have a Slack channel devoted to it. It has thus only been fitting that we use the Caltrain as a guinea pig for one of our R&D projects, which we use as a channel for exploring new technologies, comparing known ones, experimenting with new algorithms, and stress-testing hypotheses and ideas. You can currently find "Caltrain Rider" in both the Android and Apple app stores, which has the most well-engineered backend involving **[Add stuff here... Kafka queues and HBase storage. We have Spark streaming jobs continually ingesting Caltrain twitter data and analyzing it for sentiment.]**

If you have ever ridden the train, you will know that those delay estimates Caltrain provides are seemingly random. Often a train will remain "two minutes delayed" for ten minutes or longer after the train was *already* meant to have departed. Or, delays will be reported and the train will be completely on time. We have thus been working on the bigger analytical problem of how we can better predict train delays. The main input for the analysis has been data scraped from the Caltrain website (they are okay with this!). The data itself is not actually information on when a train has arrived at or left a station, but the often wrong, on-time predictions for the train, updated each minute. We interpreted that when there is no longer an on-time prediction for a train that it has actually left the station but wanted some ground truth data to support this interpretation. Taking advantage of our location next to the Caltrain tracks, we decided to point a camera at the tracks in order to visually detect when a train is passing and in what direction it is going to provide such a data point. Through this task, we learned the ease with which one can develop a motion detection algorithm, which I will share with you in this blog post. Moreover, it has also given us the chance to experiment with Raspberry Pis and with deploying and monitoring the beginnings of our own 'internet of things'.  

# Following along
In this blog post, I will walk through the basics of the algorithm we developed to identify a train's passing and direction via video. At SVDS, we are a big proponent of [Jupyter Notebooks](http://jupyter.org/) as they provide an effective method for interactively performing, documenting, visualizing, and communicating analysis as it is done. Therefore, this blog post is indeed a Jupyter Notebook and the .ipynb version can be downloaded **here** so that you can do some motion detection of your own. A helper file, `trainspotting.py`, of functions for loading different video feeds (including your own computer camera) and performing much of this analysis can also be found in the repository. 

I have hidden the actual executable code for simplicity, as it is not required to understand the algorithm development, but feel free to press the `toggle code` button at the top of this post if you would like to follow along. 

# Image and video processing with OpenCV

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2 #OpenCV
import trainspotting as ts # Some helper functions 

<img src="figures/opencv.png" alt="OpenCV" width="50" align='right'>
[OpenCV](http://opencv.org/) is one of the best packages for image processing and computer vision. It was first launched in 1999 and the third version was just released at the end of 2015. It is written in C++ but it has bindings in Python, Java, and MATLAB/OCTAVE. In this notebook, we use the Python bindings of OpenCV3. 

*Tip*: If installing for Python, I would *highly* recommend doing so through [anaconda](http://docs.continuum.io/anaconda/index) as installing it through `pip` or on other systems can be *very* painful.  


# Game plan
I will use the following video of a Caltrain passing our office to demonstrate each step to the algorithm:

In [5]:
# Load the video as a series of frames using the helper function in trainspotting.py
original = ts.read_video('video/original.mp4', display=True) 

In [13]:
"""This for loop is the base for the algorithm. 
It steps through each frame from the video and 
performs a set of actions on it. At each step, 
we will add an extra step of processing and change 
the cv2.imshow() function to output the frames 
from the most recent processing step"""
for j, frame in enumerate(original):
    
    # Show each frame in window
    cv2.imshow('Original video',frame)
    
    # Pause to allow viewing of each frame and 
    # allow window to be closed with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# Closes video window when finished 
cv2.destroyWindow('Original video')

We will approach the problem of taking such a video feed and identifying when a train is passing and its direction by breaking it down into answering the following three questions:

1. Is something moving? 
2. Is that moving thing a train? 
3. In what direction is it moving? 

The algorithm is developed to work in real time, meaning that it processes the current frame and then moves onto the next and repeats the same series of steps without any need to return to prior raw image data. Therefore, the algorithm  requires little storage (which is perfect for use on Raspberry Pis as they have very little).

# Part One: Is something moving? 

The first major step to our algorithm is determing simply if there is movement in a given frame of the video feed. We achieve this step by developing a model of what we think the stationary background in the video is and then comparing each new frame to that model to identify areas of change. This process, referred to as background subtraction, consists of processing each frame, comparing the frame to the current model of the background, identifying areas of change, and then updating the background model with the current frame.

## 1. Convert each frame to gray scale

We like to see images and video in color, which requires three pieces of information for every pixel (such as degree of red, blue and green for RGB images).  However, one channel (i.e. gray-scale) is typically all that is needed for effective background subtraction. Thus, upon reading each frame from the video feed, our first step is to convert it to gray-scale.

Numerous methods exist for converting images to one channel or gray scale including simply choosing one of the three-color channels, using weighted combinations of the blue, green, and red channels, as well as using the first component from a [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA). For this work, we will use OpenCV's gray-scale function, which uses a weighted combination of the three channels:

<center>
`gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)`
</center>

The video feed after this first phase of processing can be viewed in the following video:

In [8]:
"""This is the same for loop as seen previously for reading
and displaying the original video. However, one line has been
added that takes each frame as it is read and converts it to 
gray scale. The cv2.imshow() function has been adjusted to 
output the newly processed gray-scale frame. Running this 
cell would result in the playing of the same video as seen 
directly below."""
for j,frame in enumerate(original):
    
    # Convert frame to gray_scale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('Gray scale',gray_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Gray scale')

## 2. Smooth each frame

<img src="figures/2dgaussian.png" alt="2D Gaussian" width="150" align="right">
Our goal is to detect differences between frames (i.e. motion) but we don't want to detect any differences due to noise or small scale changes (as we are trying to detect the motion of a large train, not tiny leaves!). Therefore, to reduce noise in each frame, we apply a [Gaussian blur]
(https://en.wikipedia.org/wiki/Gaussian_blur), which replaces each pixel's value with a weighted average of it and its surrounding pixels. The weights are determined by the two-dimensional Gaussian, pictured to the right and given by the following equation: 
\begin{equation}
G(x,y) = Ae^{\frac{-(x-\mu_x)^2}{2\sigma_x^{2}}+\frac{-(y-\mu_y)^2}{2\sigma_y^{2}}}
\end{equation}
The size of the neighborhood that contributes significantly to a given pixel's new value is determined by the standard deviations, $\sigma_x$ and $\sigma_y$. Here, we will use a symmetrical Gaussian distribution, with $\sigma_x = \sigma_y$ meaning that the size of the neighborhood is equal in both the *x* and *y* direction and that we need only to provide a single parameter, `sigma_g` to the function: 
<center>
`smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_gx, sigma_gy), 0)`
</center>

Here are examples of smooth images for various standard deviations, `sigma_g`:

<img src="figures/smooth_images.png" alt="AppScreenShots" width="960">

In practice, `sigma_g` will depend on the resolution of the image, the quality of the camera, and the scale of what is trying to be detected. It will typically take iterating upon to determine the best value for a given task. 

The Gaussian blur is then applied to each frame directly after it is converted to gray-scale, as shown in the following video. In this demo, we use a `sigma_g` of 31 pixels.

In [14]:
sigma_g = 31 # Define Gaussian kernel standard deviation

"""This function builds on the last, now smoothing each 
gray-scale frame using a Gaussian blur with standard deviation
of sigma_g in both the x and y directions. The cv2.imshow()
function now outputs the smoothed image at each loop. If 
executed the same video as shown directly below would appear."""
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.
                              COLOR_BGR2GRAY)
    
    # Apply a Gaussian blur to the gray scale frame 
    smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_g, sigma_g), 0)
    
    cv2.imshow('Smooth', smooth_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
cv2.destroyWindow('Smooth')

## 3. Create model of background

As summarized earlier, our plan is to create a model of what the background of the frame is and compare each frame to it to identify areas of change. There are a number of ways to create a model of the background. 
1. Use a stationary the average pixel values of a series of frames.
2. Use the running average of a series of frames.
3. Model the background as a Gaussian mixture model.
4. Use background segmentation.
** Will say more here **

## Calculate the running average
A running average is very simple to implement and requires minimal computation (perfect for a Raspberry Pi) while at the same time is able to adapt to some changing conditions. The running average is calculated for a given pixel (*x*,*y*) at each time, *t* as:
\begin{equation}
R(x,y,t) = (1-\alpha)R(x,y,t-1) + \alpha F(x,y,t)
\end{equation}
The value $\alpha$ adjusts the weight of the current frame being averaged in and regulates the speed at which the previous frames are forgotten.

To update the running average in Python, we use:: 

```cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)```


In [11]:
alpha = 0.2 # Define weighting coefficient
running_avg = None # Define variable for running average

sigma_g = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_g, sigma_g), 0)
    
    # If this is the first frame, making running avg current frame
    # Otherwise, update running average with new smooth frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    else:
        cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    cv2.imshow('Running average', cv2.convertScaleAbs(running_avg))

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Running average')

## 4. Calculate the difference

To detect areas of change between the current frame and the current model of the background, we take the absolute difference between the running average frame and the current frame (i.e. `|smooth_frame - running_avg|`) using the [`cv2.absdiff()`](http://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#absdiff) function. 

<img src="figures/bg-fg.jpg" alt="Background and foreground" width="860">

<img src="figures/diffcolor73.png" alt="Difference frame" width="760">

Implement in Python: 

`diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))`

In [10]:
alpha = 0.2 
running_avg = None 
sigma_g = 31

for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_g, sigma_g), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
        
    # Find |difference| between current smoothed frame and running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)

    cv2.imshow('Difference', diff)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Difference')

## 5. Threshold the difference

Next, we want to consider pixels where change occurs past a given threshold. The following code sets all pixels where the current frame is more than 35 value points different the running average as being in motion and all other pixels as being still. 

<img src="figures/diffcolor73.png" alt="Difference frame" width="960">

In [12]:
thresh = 35 # All pixels with differences above this value will be set to 1
alpha = 0.2
running_avg = None 
sigma_g = 31

for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_g, sigma_g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 1, otherwise 0
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Thresholded difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Thresholded difference')

# Part two: Is that moving thing a train? 

Is enough of the frame in motion? Does this motion last as long as a train? 
* Area threshold
* Duration

In [ ]:
motion_fractions = []

thresh = 35 
alpha = 0.2
running_avg = None 
sigma_g = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (g,g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    # Calculate the fraction of pixels where motion is detected
    # i.e. where subtracted==1
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))

    motion_fractions.append(motion_fraction)
    


<img src="figures/frame-fraction-with-horizontal-and-vertical.jpg" alt="Area threshold" width="860">

In [ ]:
history_length = 50
thresh = 35 
alpha = 0.2
sigma_g = 31

frame_thresh = 0.025
running_avg = None 
history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion'])
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (g,g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))
    
    history.loc[len(history)+1] = [motion_fraction, motion_fraction>frame_thresh]
    
    detect = (history.tail(history_length).In_motion.sum()==history_length)
    
    if detect: 
        print 'Train detected beginning at frame', j-history_length
        # Reset history
        history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion']) 

# Part three: In what direction is it moving? 

<img src="figures/original73.jpg" alt="Area threshold" width="860">

## 1. Define regions of interest (ROIs)
<img src="figures/roi73c2.jpg" alt="Area threshold" width="860">

In [ ]:
for j,frame in enumerate(original):
    
    cv2.rectangle(frame, (240,270), (340,400), (0,85,167), thickness=2)
    cv2.rectangle(frame, (540,270), (640,400), (44,62,79), thickness=2)
    
    cv2.imshow('ROIs', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('ROIs')

## 2. Track activity in ROIs separately

## 3. Detect direction in real-time

1. Initialize a dataframe, `history`.
2. At each frame, *t*, add row to dataframe indicating if left and/or right meets motion threshold 
3. Repeat #2 until motion threshold met on either side for last *T* frames.
3. The side with the detection for all *T* indicates the train's direction.
4. Reinitialize the `history` dataframe to detect future trains.
5. Return to #2     
<img src="figures/left-and-right.jpg" alt="Area threshold" width="420" align='right'>
<img src="figures/train-history.png" alt="Area threshold" width="200" align='left'>


In [ ]:
history_length = 40
area_thresh = 0.05
history = pd.DataFrame(columns=['Detected left','Detected right'])

running_avg = None 
thresh = 35 
alpha = 0.2
sigma_g = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (sigma_g, sigma_g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[270:400,240:340] 
    right_area = subtracted[270:400,540:640] 
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>area_thresh, right_fraction>area_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


# Bringing it all together

The end product is a model for detecting the train given a set of frames. In practice, we must train the model on labeled training data to estimate values for the following parameters:
* `sigma_g`: gives the size of the kernel used to perform the Gaussian blur for smoothing the image. Will change according to the resolution and quality of the image and scale of the motion being detected.
* `alpha`: adjusts how much the previous frames are weighted in the `running_avg`. Will change according to frame rate of camera.
* `thresh`: is the cut-off for the value of the difference between a pixel in the current frame and moving average to be considered moving. 
* `area_thresh`: the fraction of the ROI that must be considered in motion to imply a train's passing.
* `history_length`: Number of frames train has to be detected in for it to be considered a Caltrain. Depends on frame rate and length of train to be detected. 


# Real-time deployment

## Raspberry pi setup
Cost ~ $130
<center><img src="figures/pi-hardware.png" alt="Tweent" width="900" style="horizontal-align:middle"></center>

##  Detecting motion at night

<center><img src="figures/infra.jpg" alt="Infrared photography" width="300" style="horizontal-align:middle"></center>


## PiCamera

# Conclusions

# Appendix: Trying out motion detection at home

You can try out the motion detection algorithms shown here using your own web cam! Use the following code to start. Simply press 'q' on your keyboard to end the demo.  

In [22]:
feed = cv2.VideoCapture(0)
running_avg = None
while True:
    frame = md.get_frame_cam(feed)
    smooth, running_avg = md.get_running_avg(frame, running_avg, alpha=0.02)
    diff, thresh_diff = md.get_diff(smooth, running_avg, thresh=95)
    edges, contours = md.get_edges_contours(thresh_diff)
    c_x,c_y, areas, circle_frame = md.get_centroids(contours, frame=frame)
    cv2.imshow('Frame - running average', edges)
    cv2.imshow('Threshold of frame - running average', thresh_diff)
    cv2.imshow('Edges', edges)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
feed.release()
cv2.destroyAllWindows()

# Notes from prior writing

Many motion detection algorithms would next identify the edges of entities that are moving from the threshold image, find the centroids of the polygons that the edges create, and track those centroids. However, in this case, the non-window sections of the train don't change in pixel value significantly, making the only movement that is tracked that of the windows. There are so many windows all of the same size passing by at such a very fast speed that it is difficult to track single windows entirely across the frame to detect direction of movement. 

Improvements to make traditional motion detection work would probably have required a higher resolution camera or extensive fine tuning and improvement of the image processing algorithms. Instead, we decided to take advantage of the unique characteristics of our specific problem. 